In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

### Hyper params

In [2]:
mnist = input_data.read_data_sets('../dataset/MNIST_data', one_hot=True)

Extracting ../dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../dataset/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128

In [4]:
c = 0
lr = 1e-3

In [5]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

### Geneartor

In [6]:
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [7]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

### Discriminator

In [8]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])
D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [9]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

###  Training

In [10]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])
# .astype(np.float32)

In [11]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

#### Generate data

In [12]:
G_sample = generator(Z)

In [13]:
D_real, D_logit_real = discriminator(X)

In [14]:
D_fake, D_logit_fake = discriminator(G_sample)

#### Loss

In [15]:
D_loss_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake

G_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

#### Optimize

In [16]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

#### Running

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
if not os.path.exists('out3/'):
    os.makedirs('out3/')

In [25]:
mb_size = 10000
i = 0
for it in range(100000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, 
                   feed_dict={Z: sample_Z(16, Z_dim)})
        fig = plot(samples)
        plt.savefig('out3/{}.png'.format(str(i).zfill(3)), 
                    bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], 
                  feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], 
                  feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 0.6495
G_loss: 1.903

Iter: 1000
D loss: 0.6043
G_loss: 2.192

Iter: 2000
D loss: 0.5845
G_loss: 3.203

Iter: 3000
D loss: 0.6266
G_loss: 2.292

Iter: 4000
D loss: 0.5103
G_loss: 2.827

Iter: 5000
D loss: 0.618
G_loss: 2.403

Iter: 6000
D loss: 0.6582
G_loss: 2.345

Iter: 7000
D loss: 0.5362
G_loss: 2.757

Iter: 8000
D loss: 0.6282
G_loss: 2.415

Iter: 9000
D loss: 0.6005
G_loss: 2.52

Iter: 10000
D loss: 0.5824
G_loss: 2.455

Iter: 11000
D loss: 0.5937
G_loss: 2.62

Iter: 12000
D loss: 0.5537
G_loss: 2.626

Iter: 13000
D loss: 0.5458
G_loss: 2.743

Iter: 14000
D loss: 0.4806
G_loss: 3.035

Iter: 15000
D loss: 0.5165
G_loss: 2.811

Iter: 16000
D loss: 0.5052
G_loss: 2.929

Iter: 17000
D loss: 0.4979
G_loss: 2.94

Iter: 18000
D loss: 0.4818
G_loss: 2.934

Iter: 19000
D loss: 0.5123
G_loss: 2.767

Iter: 20000
D loss: 0.4727
G_loss: 3.042

Iter: 21000
D loss: 0.4589
G_loss: 2.994

Iter: 22000
D loss: 0.4831
G_loss: 3.003

Iter: 23000
D loss: 0.4801
G_loss: 3.026

Iter: 240

### Evaluation